In [1]:
""" A genetic algorithm which evolves quantum circuits according to a goal circuit """
import random
import math
import time
import numpy as np
import qiskit.quantum_info as qi
import matplotlib.pyplot as plt
from deap import base, creator, tools
from qiskit import QuantumCircuit
from qft_circuits import *
from grover_circuits import *

def random_gate():
    """ Picks and returns a random item from the gate set
    """
    return random.choice(qpossible_gates_3)

def circuit_fitness(current_circuit, gate_set, target_matrix, num_qubits):
    """ Compares the unitary matrix from the provided circuit with the unitary
    matrix representing the target circuit
    """
    # Convert the representation of the current circuit into a QuantumCircuit
    # object
    qiskit_representation = convert_circuit(current_circuit, num_qubits, gate_set)
    # An element to element comparison between the current circuit and the target matrix
    circuit_unitary_matrix = qi.Operator(qiskit_representation)
    circuit_unitary_matrix = circuit_unitary_matrix.data
    # The circuit's fitness is the sum of the absolute element to element difference
    fitness = 0
    for i in range(0, len(circuit_unitary_matrix)):
        for j in range(0, len(circuit_unitary_matrix[i])):
            fitness += abs(circuit_unitary_matrix[i][j] - target_matrix[i][j])

    # Deap requires all fitness functions to return a tuple (even single objective functions)
    # The size of the circuit is also returned to enbale the EA to be multi-objective
    # as such that it minimises the size of the circuits in the population as well
    return (fitness,)

def circuit_size(current_circuit):
    """Calculates the 'size' of a provided circuit, where this is the
    number of gates in the circuit.
    """
    size = 0
    for gate in current_circuit:
        if gate[0] != 10:
            size += 1

    return size

def convert_circuit(current_circuit, num_qubits, gate_set):
    """ Converts a given solution into a Qiskit QuantumCircuit object
    """
    # Creates a new QuantumCircuit object to add the values to
    circuit = QuantumCircuit(num_qubits)
    for gate in current_circuit:
        # If the current gate is anything but a wire, decompose the gate and add it to the qiskit quantum circuit
        if gate_set[gate[0]] != 'WIRE':
            circuit.append(
                gate_set[gate[0]],
                gate[1]
            )

    return circuit

def mutate(circuit):
    """ Mutates a gate in the given circuit by changing it to another gate from the provided gate set at random (valid gates only?) 
    """
    # Declare the set of possible valid mutations
    possible_gates = qpossible_gates_3
    
    # Choose a random gate in the circuit via index
    mutation_index = random.randint(0, len(circuit) - 1)
    # Choose a random gate from the gate set to replace said gate
    circuit[mutation_index] = random.choice(possible_gates)

    # Need to delete the mutated individuals fitness values as they are no
    # longer related to the individual
    del circuit.fitness.values

    return circuit    

def crossover(circuit1, circuit2):
    """ Performs an inplace two/ multipoint crossover between two circuits
    """
    # TODO: Crossover is applied with probability 0.7
    # For the minimum viable product 2-point crossover is used
    # For the final product use a randomly selected multi-point crossover
    num_points = 2
    # Choose num_points random indcies to swap
    # If there is an odd number of points selected, the end of the list is
    # chosen as the end point for the second crossover
    if num_points%2 != 0:
        indicies = []
        indicies.append(len(circuit1) - 1)
    else:
        indicies = []
    
    # Validate that no two crossover points are the same
    for i in range(0, num_points):
        temp_index = random.randint(0, len(circuit1) - 1)
        while temp_index in indicies:
            temp_index = random.randint(0, len(circuit1) - 1)

        indicies.append(temp_index)

    # Sort the list of indexes so that the crossover can be performed correctly
    indicies.sort()

    # Perform the crossover in place, swapping all values between the crossover points
    for i in range(0, len(indicies), 2):
        temp_values = circuit1[indicies[i]:indicies[i+1]]
        circuit1[indicies[i]: indicies[i+1]] = circuit2[indicies[i]: indicies[i+1]]
        circuit2[indicies[i]: indicies[i+1]] = temp_values

    # Delete the fitness values associated with the "mated" circuits
    # As they are no longer related to the individual
    del circuit1.fitness.values
    del circuit2.fitness.values

    return (circuit1, circuit2)  
        
def tournament_selection(population, k, tournament_size, gate_set, target_matrix):
    """ Executes a tournament (of size 'tournament_size') based selection on the population provided,
    performing k tournaments and returning the winners in a list.
    """
    victors = []

    for i in range(0, k):
        # The fittest value starts as infinity as lower fitness values are desired
        fittest = math.inf
        # Stores the representation of the circuit to add
        circuit_to_add = 0
        # Select size individuals randomly from the population
        for j in range(0, tournament_size):
            individual_to_compete = random.choice(population)
            current_fitness, = circuit_fitness(individual_to_compete, gate_set, target_matrix, 4)
            if current_fitness < fittest:
                fittest = current_fitness
                circuit_to_add = individual_to_compete

        victors.append(circuit_to_add)
        # Reset the fitness tracker
        fittest = 0
        # Don't need to remove the chosen circuit from the population (nothing
        # to stop it from being chosen more than once)

    return victors

# Create the gate set (actually a dictionary where the key value pair is indexes and the Qiskit gate name)
# The number of qubits supplied here is the same as the number of quibts being looked at in the rest of the program

# Can't hardcode the array values for the target unitary matrix as the values are rounded implicitly

# Not decomposing the goal circuit as it seems to have at least some affect on the values stored in the unitary matrix representing the circuit
CIRCUIT_TYPE = "qft"
# QFT circuits only need to decomposed once, whereas Grover's algorithm circuits need to be decomposed twice
if CIRCUIT_TYPE == "qft":
    # The two qubit QFT circuit is being created in this instance
    goal_circuit = qft_circuit3
    gate_set = qgate_set3
    goal_circuit.decompose().draw(output="latex", filename="test_circuit.pdf")
    goal_matrix = qi.Operator(goal_circuit)
    goal_matrix = goal_matrix.data
    goal_circuit = qft_circuit3.decompose()
elif CIRCUIT_TYPE == "grover":
    # The two qubit Grover circuit is being created in this instance
    goal_circuit = grover_circuit3
    gate_set = ggate_set3
    goal_circuit.decompose().decompose().draw(output="latex", filename="test_circuit.pdf")
    goal_matrix = qi.Operator(goal_circuit)
    goal_matrix = goal_matrix.data
    goal_circuit = grover_circuit3.decompose().decompose()

# Create a basic minimising fitness object (This should attempt to minimise the difference between the unitary matrices between circuits)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# Create a basic object for each inidivdual in the population (an empty list) with a fitness linked to the above fitness object
creator.create("Individual", list, fitness=creator.FitnessMin)

# The max length of a circuit (the maximum number of gates in the circuit)
# This is set to 1 + the length of the desired circuit to allow for alternative circuits to be made
CIRCUIT_LENGTH = goal_circuit.decompose().size() + 1

toolbox = base.Toolbox()
# The genes in the genome (objects that make up an individual)
toolbox.register("attribute_gate", random_gate,)
# Second argument for the register method, this redirects to the initRepeat
# function, creating an individual and repeatedly (CIRCUIT_LENGTH times)
# filling the circuit with random gates
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute_gate, n=CIRCUIT_LENGTH)
# A bag population (one without any ordering) is used and regsitered with the toolbox accordingly
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Using the toolbox to register tools instead of using them independently helps
# keep the rest of the algorithms independent from the operator set - also
# makes it easier to locate and change any tools in the toolbox
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
# toolbox.register("select", tournament_selection)
# The NSGA II algorithm is used for selection as it is multi-objective
# Only 95 individuals are chosen as the remaining 5 are provided from intial elitist selection
toolbox.register("select", tools.selTournament)
toolbox.register("evaluate", circuit_fitness, gate_set=gate_set, target_matrix=goal_matrix, num_qubits=4)
# The use of the toolbox allows for algorithms that resemble pseudocode as closely as possible (as generic as possible)

# Register the statistics object to track the statistics/ progress of the genetic algorithm
statistics_fitness = tools.Statistics(key=lambda ind: ind.fitness.values[0])
statistics_size = tools.Statistics(key=circuit_size)
m_statistics = tools.MultiStatistics(fitness=statistics_fitness, size=statistics_size)
m_statistics.register("average", np.mean)
# To cut down on statistical compilation time, this statistic has been excluded
# m_statistics.register("standard deviation", np.std)
m_statistics.register("minimum", np.min)
m_statistics.register("maximum", np.max)

logbook = tools.Logbook()

[CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 1),), clbits=()), CircuitInstruction(operation=Instruction(name='cp', num_qubits=2, num_clbits=0, params=[1.5707963267948966]), qubits=(Qubit(QuantumRegister(2, 'q'), 1), Qubit(QuantumRegister(2, 'q'), 0)), clbits=()), CircuitInstruction(operation=Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0),), clbits=()), CircuitInstruction(operation=Instruction(name='swap', num_qubits=2, num_clbits=0, params=[]), qubits=(Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)), clbits=())]
16
26
70


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/qiskit/visualization/circuit/latex.py:154: FutureWarning: The default matplotlib drawer scheme will be changed to "iqp" in a following release. To silence this warning, specify the current default explicitly as style="clifford", or the new default as style="iqp".
  self._style, _ = load_style(style)


In [2]:
def main(num_generations, population_size, mutation_rate, crossover_rate, elitism_rate):
    """ The main program function that executes the genetic algorithm
    """
    # The genetic algorithm will run for the given number of iterations, or until the desired circuit is reached
    NUM_GENERATIONS = num_generations
    POP_SIZE = population_size
    MUTATION_RATE = mutation_rate
    CROSSOVER_RATE = crossover_rate
    ELITISM_RATE = elitism_rate
    # Create an initial population of 100 circuits (individual circuits stored as a list)
    population = toolbox.population(n=POP_SIZE)

    # Stores the genetic representation and fitness values of the best solution found thus far
    # Initialised with placeholder values
    best_solution = [math.inf, 0]

    # Validate the fitness value of all circuits (this is done so any elite circuits carried over from the first generation have a valid fitness.values attribute)
    for circuit in population:
        circuit.fitness.values = circuit_fitness(circuit, gate_set, goal_matrix, 4)

    for gen in range(0, NUM_GENERATIONS):
        # Select the individuals to be used in the current generation
        offspring = toolbox.select(population, 100, 3)
        # Clone the selected population so that it can be altered
        offspring = list(map(toolbox.clone, offspring))

        next_gen_population = []
        # Sort the offspring so that the fittest circuits can be found
        for i in range(0, len(population)):
            fitness, = circuit_fitness(population[i], gate_set, goal_matrix, 4)
            next_gen_population.append((fitness, population[i]))

        next_gen_population.sort()
        # If the best solution is better than the current, replace it
        if next_gen_population[0][0] < best_solution[0]:
            best_solution[0] = next_gen_population[0][0]
            best_solution[1] = next_gen_population[0][1]

        temp_list = []
        # Add the ELITISM_RATE best solutions from the population to the temp list (remove their fitness value from the tuple)
        for i in range(0, ELITISM_RATE):
            temp_list.append(next_gen_population[i][1])
        
        # Reset the population for the next generation so that only the ELITISM_RATE best
        # circuits are copied over
        next_gen_population = temp_list

        # Apply crossover to this generation of circuits
        for c1, c2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CROSSOVER_RATE:
                toolbox.mate(c1, c2)
                # Delete the fitness values from the parent circuits as they have been
                # altered (value no longer corresponds to the circuits actual fitness)
                del c1.fitness.values
                del c2.fitness.values
        
        # Apply mutation to this generation of circuits
        for child in offspring:
            if random.random() < MUTATION_RATE:
                toolbox.mutate(child)
                # For the aforementioned reason, delete the circuits fitness value
                del child.fitness.values

        # Revalidates the fitness of individuals that have an invalid fitness
        # First find and store all the circuits that have an invalid fitness value
        altered_circuits = []
        for child in offspring:
            if not child.fitness.valid:
                altered_circuits.append(child)
        
        fitnesses = toolbox.map(toolbox.evaluate, altered_circuits)
        for (circuit, fitness) in zip(altered_circuits, fitnesses):
            circuit.fitness.values = fitness

        # Randomly pick circuits from the offspring to fill up the rest of the population to be used in the next generation
        while len(next_gen_population) < len(population):
            chosen_circuit = random.choice(offspring)
            next_gen_population.append(chosen_circuit)
            # Remove the circuit, so there is less chance duplicate individuals end up in the next generation
            offspring.remove(chosen_circuit)    

        # The population is updated
        population[:] = next_gen_population
        record = m_statistics.compile(population)
        logbook.record(**record)
    
    return (best_solution[0], circuit_size(best_solution[1]))

In [3]:
if __name__ == "__main__":
    # For the experiments process, the results of each run aren't plotted on a graph but are instead stored in a the crossover_results.txt file
    # For each crossover rate, the evolutionary algorithm is run 10 times
    cx_prob = 0
    while cx_prob <= 1:
        # Stores the sum of the fitness, size and execution time of all runs
        # for a given crossover rate to enable the average to be calculated
        fitness_sum = 0
        size_sum = 0
        times_sum = 0

        for run in range(0, 10):
            # Executes and times the run
            start = time.time()
            best_fitness, best_size = main(100, 100, max(1/CIRCUIT_LENGTH, 0.4), cx_prob, math.floor(0.05 * 100))
            end = time.time()
            fitness_sum += best_fitness
            size_sum += best_size
            times_sum += round(end-start, 5)
       
        # Logs the average values over a crossover rate into the aforementioned file
        file = open("experiment_results/crossover_results.txt", "a")
        data = str(round(fitness_sum/10, 5)) + "," + str(round(size_sum/10, 5)) + "," + str(round(times_sum/10, 5)) + "," + str(cx_prob) + "\n"
        file.write(data)
        file.close()
        
        cx_prob += 0.1